# Destiny

#### Importando pacotes e baixando banco de dados disponibilizado pela CVM

In [ ]:
#Instalando as bibliotecas 
# %pip install pandas 
# %pip install numpy
# %pip install imblearn
# %pip install lazypredict
# %pip install scikit-learn


In [ ]:
# Bibliotecas de Manipulação de Dados e Visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas de Machine Learning e Estatísticas
from imblearn.over_sampling import SMOTE
from lazypredict.Supervised import LazyClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:
#Lendo arquivo csv que será utilizado como banco de dados
df = pd.read_csv('.\dataset\IM_semNP.csv')

#### Eliminando numeros nulos e duplicatas

In [ ]:
#Eliminando todas as linhas duplicadas e vazias 
df = df.dropna().drop_duplicates()

#### Separação de Features

In [ ]:
#criação de uma variavel apenas com as colunas que serão utilizadas no modelo
Colunas_Quero = [ 'Nome_Fundo',
                   'Fundo_Exclusivo', 'Ativo', 'Ativo_Carteira', 'Ativo_Direitos_Aquisicao', 
                   'Ativo_Direitos_Aquisicao_Creditos_Vencer_Adimplentes', 'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes', 'Ativo_Direitos_Sem_Aquisicao',
                   'Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Adimplentes', 
                   'Ativo_Direitos_Sem_Aquisicao_Creditos_Vencer_Inadimplentes', 
                   'Ativo_Valores_Mobiliarios', 'Patrimonio_Liquido', 'Liquidez_Ate_30_Dias', 
                   'Liquidez_Ate_60_Dias', 'Liquidez_Ate_90_Dias', 'Liquidez_Ate_180_Dias', 'Liquidez_Ate_360_Dias',
                   'Liquidez_Acima_360_Dias','Carteira', 'Ativo_Direitos_Aquisicao_Parcelas_Inadimplentes', 
                   'Ativo_Direitos_Aquisicao_Creditos_Inadimplentes','Ativo_Direitos_Aquisicao_Creditos_Vencer_Inadimplentes',"Carteira_Direitos_Aquisicao_Inadimplentes"
                   ]

In [ ]:
#Excluindo colunas que não serão utilizadas no modelo
todas_colunas2 = list(df.columns)
for coluna in todas_colunas2:
    if coluna not in Colunas_Quero:
        df = df.drop(coluna,axis=1)

#### Cálculo da taxa de inadimplência

In [ ]:
#cálculo e criação da coluna taxa de inadimplência
df["Taxa_Inadimplencia"] = df["Carteira_Direitos_Aquisicao_Inadimplentes"]/df["Patrimonio_Liquido"]

#### Agrupando as features em prazos

In [ ]:
#divisão do prazo de liquidez entre curto, médio e longo prazo
df['Liquidez_curtoPrazo'] = df['Liquidez_Ate_30_Dias'] + df['Liquidez_Ate_60_Dias']
df = df.drop(['Liquidez_Ate_30_Dias','Liquidez_Ate_60_Dias'],axis=1)
df['Liquidez_medioPrazo'] = df['Liquidez_Ate_90_Dias'] + df['Liquidez_Ate_180_Dias']
df = df.drop(['Liquidez_Ate_90_Dias','Liquidez_Ate_180_Dias'],axis=1)
df['Liquidez_longoPrazo'] = df['Liquidez_Ate_360_Dias'] + df['Liquidez_Acima_360_Dias']
df = df.drop(['Liquidez_Ate_360_Dias','Liquidez_Acima_360_Dias'],axis=1)

#### Transformando features categóricas em numéricas

In [ ]:
#Transformando os dados da coluna fundo exclusivo em dados numericos 
df['Fundo_Exclusivo'] = df.Fundo_Exclusivo.replace({'Não':0, 'Sim':1})

#### Criando Gráficos

In [ ]:
# Selecionando apenas as colunas numericas
colunas_numericas = []

for coluna in df.columns:
        if pd.api.types.is_numeric_dtype(df[coluna].dtype):
            colunas_numericas.append(coluna)

df_numerico = df[colunas_numericas]

In [ ]:
# Criando a matriz de correlação das colunas
correlation = df_numerico.corr()
plt.figure(figsize=(14, 10))
plot = sns.heatmap(correlation, annot = True, fmt=".1f", linewidths=.1)
plot

In [ ]:
#Criando gráfico da análise da taxa de inadimplência
plt.figure(figsize=(10, 7))
plt.grid(True)
plt.title("Analise da taxa de inadimplência")
plt.xlabel("Index dos Fundos")
plt.ylabel("Taxa de inadimplência")

valores_y = [i * 1.5 for i in range(-50, 50)]
plt.yticks(valores_y)

plt.plot(df.index, df["Taxa_Inadimplencia"], marker='o', linestyle='-', color='b', label='Dados de Exemplo')

plt.show()



#### Normalizando as colunas

In [ ]:
# Normalizando as colunas numéricas 
scaler = MinMaxScaler()
todas_colunas = list(df_numerico.columns)
df[todas_colunas] = scaler.fit_transform(df[todas_colunas])

#### Clusterização

Tratamento dos dados para clusterização

In [ ]:
# Selecionando apenas os fundos que possui taxa de inadimplência maior que 0
for index, linha in df.iterrows():
    if linha['Taxa_Inadimplencia'] < 0:
        df = df.drop(index, axis=0)


Fazendo o algorítimo para clusterização

In [ ]:
# 1. Tratar os Valores Ausentes
data = df[['Taxa_Inadimplencia']]
data = data.fillna(data.mean())  # Preenche os valores NaN com a média da coluna

# 2. Aplicar o K-means
K = 2
kmeans = KMeans(n_clusters=K, random_state=42)
kmeans.fit(data)


In [ ]:
labels = kmeans.labels_
df['Cluster'] = labels

#### Buscando nomes de fundos saudáveis e não saudáveis

In [ ]:
# Verificando o desbalanceio dos dados
df['Cluster'].value_counts()

In [ ]:
# Criando um gráfico de dispersão dos fundos
plt.scatter(df['Taxa_Inadimplencia'], np.zeros_like(df['Taxa_Inadimplencia']),c=df['Cluster'])
plt.xlabel('Taxa de inadimplencia')
plt.ylabel([])
plt.title('Gráfico de Dispersão dos Clusters')
plt.show()

#### Separando dados para treino e teste

In [ ]:
# Criando um novo dataframe apenas com as colunas para o terinamento
df_treinamento = df.drop('Nome_Fundo', axis=1)

# Exclui dados vazios
df_treinamento.dropna(inplace=True)

X = df_treinamento.copy()
X = X.drop("Cluster", axis=1)
X = pd.DataFrame(X)

Y = df_treinamento["Cluster"]
Y = Y.values.reshape(-1, 1)
Y = pd.DataFrame(Y)

Train_X, Test_X, Train_Y, Test_Y = train_test_split(X,Y, test_size=0.3)


#### Resolvendo os dados desbalanceados

In [ ]:
#instanciando um objeto
smote = SMOTE(random_state = 32)

# Balanceando os dados de treinos, para ter a mesma quantidade de fundos saudaveis e não saudaveis
X_smote_res, Y_smote_res = smote.fit_resample(Train_X, Train_Y)

#### Utilizando Lazy predict para analisar os melhores modelos

In [ ]:
#Analise os melhores modelos com o Lazy predict
clf = LazyClassifier()
models = clf.fit(Train_X, Test_X, Train_Y, Test_Y)
print(models)

#### Descobrindo overfitting

In [ ]:
display(models)

#### Random Forest Classifier

In [ ]:
# Treinando o RandomForest
rf = RandomForestClassifier(n_estimators=100, random_state=0, oob_score=True, max_features=8)

rf.fit(X_smote_res, Y_smote_res)
pred_rf = rf.predict(Test_X)

In [ ]:
pred_rf = pd.Series(pred_rf)

In [ ]:
# Analisando os acertos, do modelo
acc = accuracy_score(Test_Y, pred_rf) #Verificando a acurácia do modelo
print(f'Acurácia: {acc:.2f}')

f1 = f1_score(Test_Y, pred_rf, average='weighted') #Verificando o F1 score do modelo
print(f'f1_score {f1:.2f}')


#### Analise fundos quebrados

##### Separação dos nomes dos fundos quebrados

In [ ]:
#Lista com nomes de fundos que sabemos que quebraram
fundos_quebrados = ['FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS INDIGO BARTER',
                    'SB CRÉDITO FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS MULTISSETORIAL',
                    'CAPTALYS FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS - MAIS LOTES',
                    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS BRAVA CHALLENGE',
                    'CREDIHOME FUNDO DE INVESTIMENTO EM DIREITOS CREDITORIOS',
                    'SAFIRA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS',
                    'RUBI FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS MULTISETORIAL',
                    'FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS BULLLA',
                    'LS INTERBANK FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',
                    'MANGALARGA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO-PADRONIZADOS',
                    'TURQUESA - FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS',
                    'ÔNIX FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADO',
                    'RAVENNA FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS NÃO PADRONIZADOS',
                    ]

##### criação de tabela com os fundos quebrados


In [ ]:
#Criação de tabela apenas com os fundos quebrados
df_fq = df[df['Nome_Fundo'].isin(fundos_quebrados)]

In [ ]:
#Criação do valor em risco 
df_fq["Valor_em_Risco"] = df_fq["Ativo_Direitos_Aquisicao_Creditos_Vencer_Inadimplentes"]/df_fq["Patrimonio_Liquido"]
df_fq1 = df_fq.loc[(df_fq.Valor_em_Risco > 0)]

##### analise do valor de risco dos fundos quebrados

In [ ]:
#Criação de grafico com análise do valor em risco dos fundos quebrados
plt.figure(figsize=(10, 7))
plt.grid(True)
plt.title("Analise do valor em risco do fundos")
plt.xlabel("Index dos Fundos")
plt.ylabel("Valor em risco")

valores_y = [i * 0.50 for i in range(0, 50)]
plt.yticks(valores_y)

plt.plot(df_fq1.index, df_fq1["Valor_em_Risco"], marker='o', linestyle='-', color='b', label='Dados de Exemplo')

plt.show()

##### clusterização dos fundos quebrados


In [ ]:
#Clusterizando fundos quebrados
df_fqt = df_fq[['Ativo', 'Valor_em_Risco']].copy()

for index, linha in df_fqt.iterrows():
    if linha['Valor_em_Risco'] == 0:
        df_fqt = df_fqt.drop(index, axis=0)

df_fqt.dropna(subset=['Ativo', 'Valor_em_Risco'], inplace=True)

In [ ]:
#Separando dados para clusterização dos fundos quebrados
if df_fqt.empty:
    print("O DataFrame está vazio. Verifique seus dados.")
else:

    dados = df_fqt[['Ativo', 'Valor_em_Risco']].values


    if len(dados) >= 2:  
        K = 4
        kmeans = KMeans(n_clusters=K, init='k-means++', random_state=42)
        kmeans.fit(dados)

        cluster_labels = kmeans.labels_
        centroids = kmeans.cluster_centers_

        print(f"Labels dos Clusters: {cluster_labels}")
        print(f"Centróides dos Clusters: {centroids}")
    else:
        print("Não há amostras suficientes para aplicar o K-Means.")

In [ ]:
dados = df_fqt[['Ativo', 'Valor_em_Risco']].values

K = 4
kmeans = KMeans(n_clusters=K, init='k-means++', random_state=42)
kmeans.fit(dados)

cluster_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [ ]:
#Criação de gráfico com a disperção dos clusters dos fundos quebrados
plt.scatter(df_fqt['Ativo'], df_fqt['Valor_em_Risco'], c=cluster_labels)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', c='red', s=100)
plt.xlabel('Ativo')
plt.ylabel('Valor em Risco')
plt.title('Gráfico de Dispersão dos Clusters')
plt.show()